<a href="https://colab.research.google.com/github/SaumyaKarnwal/DeepFake-Detection-in-Government-Documents/blob/main/CV_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'deepfake-and-real-images:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1909705%2F3134515%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240304%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240304T121800Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3377a0cdd05a0153ce21a8d7625ba1ddb857654cdee5cba79bbb7c496676cac5d6d4a5455e5af828386921093b0a5bea6acf5039047dea2654192ce3a1871e2d239e34dacb1a62e32a21283c2997167681f9e862d59ac263625a86e17cbb7cd1dc72b0bbd10f673a772d8f4fd78a098ed43d42dc03ae2c0ab855f520b69882913b15c87893d5ec44eb928d912d4561ac5346a6de6a6b95ea61eed596b7814960bd353255a88000c829e625196316be921f3d981c5ae5e5ef6764b7ebb0492a61478077d18839c7a0324cff95ee8a1a8a454c854df0555a663fba2757f98d58b83e9a944898a858a76252f33475beae5d0b8735b5fd7bfccb89b96c95a9a59ab4,modelbestx:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4428637%2F7606397%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240304%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240304T121800Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1efd349d292f682de614547bf488adb4ce5dfb2431bf4a1f1164a58ae63665afe8809e4d8910b907fb9838fca994f14418a2deb285d68e515bff3b6cd77fd6d12d4ea57b6f801e764992614de6cd5473342109bca00115671739deeffe06ff958c150745f2b95ebfe18b2fbfb24b47ce9c3818c3fdb4bdc62ffbdc7fab59618281c88b911c48b32dd9a3aa06dffe35a679d0b8818c06ceaf93f6743c226fa841edcc66283adf864fa58f4a37fd367b971bd5fe34d0ad30fbb7b1e474aaa56f3332db41a6d3af190ce9f25cf285a563bfc3b9f07d1fde1aa62a0079f5bf9da7ce349f8cc20db79f76f42d92f4a3e4e45b8ee62568348fc363c3e73207dd08fe6a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1808722718 bytes downloaded
Downloaded and uncompressed: deepfake-and-real-images
[==================================================] 5652716 bytes downloaded
Downloaded and uncompressed: modelbestx
Data source import complete.


## Importing Libraries

In [ ]:
import tensorflow as tf
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
import numpy as np

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.22 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 28.4/107.7 GB disk)


In [ ]:
import cv2
from ultralytics import YOLO
import os

In [ ]:

# Set the path to your dataset
train_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Train'
validation_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Validation'
test_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Test'


## Preprocessing the images

### CS LBP

In [ ]:
# Define the function to calculate CS-LBP for a given pixel
def get_pixel(img, x1, y1, x, y):
    new_value = 0
    try:
        if img[x1][y1] >= img[x][y]:
            new_value = 1
    except IndexError:
        pass
    return new_value

def cs_lbp_calculated_pixel(img, x, y):
    val_ar = []

    val_ar.append(get_pixel(img, x, y+1, x, y-1))
    val_ar.append(get_pixel(img, x+1, y+1, x-1, y - 1))
    val_ar.append(get_pixel(img, x+1, y, x-1, y))
    val_ar.append(get_pixel(img, x+1, y-1, x - 1, y + 1))

    power_val = [1, 2, 4, 8]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val

### CLAHE

In [ ]:
# Define the function to apply CS-LBP and CLAHE to an image
def apply_cs_lbp_clahe(image, output_directory, label, clip_limit=2.0, grid_size=(8, 8)):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size).apply(img_gray)
    height, width = img_clahe.shape
    img_cs_lbp = np.zeros((height, width), np.uint16)

    for i in range(0, height):
        for j in range(0, width):
            img_cs_lbp[i, j] = cs_lbp_calculated_pixel(img_clahe, i, j)

    # Save the CS-LBP image
    cv2.imwrite(os.path.join(output_directory, label), img_cs_lbp)

    #plt.imshow(img_cs_lbp, cmap="gray")
    #plt.title("CS-LBP Image")
    #plt.show()


### YOLO

In [ ]:
# Load YOLO model
model = YOLO("/kaggle/input/modelbestx/best.pt")

# Function to extract face ROI from an image using YOLO
def extract_face(img_path):
    # Perform object detection
    results = model(img_path)
    boxes = results[0].boxes
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if len(boxes) > 0:
        # Crop and save faces, and display them
        for i, box in enumerate(boxes):
            top_left_x = int(box.xyxy.tolist()[0][0])
            top_left_y = int(box.xyxy.tolist()[0][1])
            bottom_right_x = int(box.xyxy.tolist()[0][2])
            bottom_right_y = int(box.xyxy.tolist()[0][3])

            # Crop face region
            roi = img[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
            return roi
    else:
        return None

clip_limit = 2.0
grid_size = (8, 8)

### Applying YOLO, CLAHE, CS LBP

In [ ]:
# Define the function to process each dataset
def process_dataset(real_dir, fake_dir, output_parent_dir):
    for label, directory in zip(['Real', 'Fake'], [real_dir, fake_dir]):
        output_dir = os.path.join(output_parent_dir, label)
        os.makedirs(output_dir, exist_ok=True)

        # Process each image in the directory
        for image_file in os.listdir(directory):
            image_path = os.path.join(directory, image_file)
            face_roi = extract_face(image_path)
            if face_roi is not None:
                apply_cs_lbp_clahe(face_roi, output_dir, image_file)

# Set the paths for each dataset
train_real_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Train/Real'
train_fake_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Train/Fake'
test_real_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Test/Real'
test_fake_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Test/Fake'
validation_real_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Validation/Real'
validation_fake_dir = '/kaggle/input/deepfake-and-real-images/Dataset/Validation/Fake'
output_dir = '/kaggle/working/cslbp_images_clahe'

# Process each dataset
process_dataset(train_real_dir, train_fake_dir, os.path.join(output_dir, 'training'))
process_dataset(test_real_dir, test_fake_dir, os.path.join(output_dir, 'testing'))
process_dataset(validation_real_dir, validation_fake_dir, os.path.join(output_dir, 'validation'))

print("CS-LBP transformation with CLAHE completed for all images in the train, validation, and test datasets.")


image 1/1 /kaggle/input/deepfake-and-real-images/Dataset/Train/Real/real_13992.jpg: 640x640 (no detections), 1049.8ms
Speed: 46.8ms preprocess, 1049.8ms inference, 23.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/deepfake-and-real-images/Dataset/Train/Real/real_17406.jpg: 640x640 2 FACEs, 424.4ms
Speed: 16.1ms preprocess, 424.4ms inference, 21.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/deepfake-and-real-images/Dataset/Train/Real/real_43796.jpg: 640x640 3 FACEs, 275.4ms
Speed: 6.5ms preprocess, 275.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/deepfake-and-real-images/Dataset/Train/Real/real_69788.jpg: 640x640 (no detections), 243.4ms
Speed: 5.6ms preprocess, 243.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/input/deepfake-and-real-images/Dataset/Train/Real/real_29752.jpg: 640x640 1 FACE, 239.8ms
Speed: 5.0ms preprocess, 239.8ms infe

KeyboardInterrupt: 

In [ ]:
# Set the path to your dataset
train_dir = '/kaggle/working/cslbp_images_clahe/training'
validation_dir = '/kaggle/working/cslbp_images_clahe/validation'
test_dir = '/kaggle/working/cslbp_images_clahe/testing'

### Data Augmentation and Normalization

In [ ]:
# Define data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


### Resizing the Images

In [ ]:
# Use datagen.flow_from_directory() to load and preprocess your dataset
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',  # Assuming a binary classification task (fake vs real)
    classes=['Fake','Real']  # Specify the class names

)


Found 140002 images belonging to 2 classes.


In [ ]:
validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    classes=['Fake', 'Real']
)

Found 39428 images belonging to 2 classes.


In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    classes=['Fake', 'Real']
)

Found 10905 images belonging to 2 classes.


## Training the model

### Compiling the CNN

In [ ]:
# Define the model
model = Sequential()

# Convolutional Layer 1
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

# Convolutional Layer 2
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Convolutional Layer 3
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Fully Connected Layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Display the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 224, 224, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 74, 74, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 74, 74, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 74, 74, 64)        2

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN

In [ ]:
# Train the model
model.fit(train_generator, epochs=40, validation_data=validation_generator)


Epoch 1/40


KeyboardInterrupt: 

In [ ]:
# Create the output directory if it doesn't exist
output_dir = '/kaggle/output/'
os.makedirs(output_dir, exist_ok=True)

# Save the model to the output directory
model.save(os.path.join(output_dir, 'final_model.keras'))


In [ ]:
from keras.models import load_model

# Replace 'final_model.h5' with the actual file name you used to save your model
saved_model_path = '/kaggle/output/final_model.keras'

# Load the saved model
loaded_model = load_model(saved_model_path)

# Now you can use 'loaded_model' for inference or further training


KeyboardInterrupt: 

### Testing the Performance of the Model

In [ ]:
from keras.preprocessing import image
import numpy as np

# Define the path to the new image you want to classify
new_image_path = '/kaggle/working/extracted_faces/Test/Fake/fake_3300.jpg'

# Load the image and preprocess it for the model
img = image.load_img(new_image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Normalize the pixel values

# Make predictions using the loaded model
predictions = loaded_model.predict(img_array)

# Assuming binary classification (real vs. fake)
# You may have to adjust this based on your model's output
if predictions[0, 0] > 0.5:
    print("Prediction: Real Image")
else:
    print("Prediction: Fake Image")


In [ ]:
# Evaluate the model on the test_generator data
evaluation = loaded_model.evaluate(test_generator)

# The evaluate method returns the loss and accuracy
loss, accuracy = evaluation

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')